## Pre Process


## Post Process


#### init function

In [3]:
import os, sys, random

In [1]:
import subprocess
from lib.library import delete_similar_image_in_subfolders, add_folder_name_to_files, rename_subfolders

def tagging_image(working_folder):
    command = [
        r"D:\Project\Code\Diffusion_Data_Tools\myenv\Scripts\accelerate.EXE", "launch",
        r"D:\Project\Code\Diffusion_Data_Tools\lib\tag_images_by_wd14_tagger.py",
        "--batch_size", "1",
        "--caption_extension", ".txt",
        "--caption_separator", ",",
        "--character_threshold", "0.5",
        "--max_data_loader_n_workers", "2",
        "--onnx",
        "--recursive",
        "--remove_underscore",
        "--repo_id", "SmilingWolf/wd-v1-4-convnextv2-tagger-v2",
        working_folder,
    ]

    subprocess.run(command)


#### main runner


In [2]:
working_folder = r'D:\Project\CivitAI\Disney\Princess\Test\draft'  

delete_similar_image_in_subfolders(working_folder)
tagging_image(working_folder)
add_folder_name_to_files(working_folder, tag_dropping_rate = 0.3)
rename_subfolders(working_folder)

Processing folder: 4_sprite
                                                                                       100% |███████████████████| 17/17 [49.0ms elapsed, 0s remaining, 347.2 samples/s]     
                                                                               100% |███████████████████| 17/17 [7.0s elapsed, 0s remaining, 2.4 samples/s] 
Initial image count: 17, Images retained: 17, Images deleted: 0
Processing folder: mulan
                                                                                       100% |███████████████████| 14/14 [50.2ms elapsed, 0s remaining, 278.7 samples/s]     
                                                                                   100% |███████████████████| 14/14 [496.9ms elapsed, 0s remaining, 28.6 samples/s] 
Initial image count: 14, Images retained: 14, Images deleted: 0


#### create prompt

In [14]:
def get_text_files(directory):
    text_files = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(".txt"):
                text_files.append(os.path.join(root, file))
    return text_files

def create_output_file(file_counter, out_path):
    output_file = os.path.join(out_path, f'test_prompt_{file_counter}.txt')
    open(output_file, 'w').close()
    return output_file


def process_file(txt_file, line_counter, file_counter, output_file, max_num_lines, suffix):
    with open(txt_file, 'r') as infile:
        for line in infile:
            line_counter, file_counter, output_file = process_line(
                line, line_counter, file_counter, output_file, max_num_lines, suffix
            )
    return line_counter, file_counter, output_file

def process_subdir(sub_dir, line_counter, file_counter, output_file, max_num_lines, prompt_per_folder, suffix):
    text_files = get_text_files(sub_dir)
    if len(text_files) >= prompt_per_folder:
        selected_files = random.sample(text_files, prompt_per_folder)
        for txt_file in selected_files:
            line_counter, file_counter, output_file = process_file(
                txt_file, line_counter, file_counter, output_file, max_num_lines, suffix
            )
    return line_counter, file_counter, output_file

def process_directory(directory, max_num_lines, prompt_per_folder=2, out_path="", suffix=""):
    file_counter = 1
    line_counter = 0
    output_file = create_output_file(file_counter, out_path)

    for root, dirs, _ in os.walk(directory):
        for dir in dirs:
            sub_dir = os.path.join(root, dir)
            line_counter, file_counter, output_file = process_subdir(
                sub_dir, line_counter, 
                file_counter, output_file, 
                max_num_lines, prompt_per_folder, 
                suffix
            )

def process_line(line, line_counter, file_counter, output_file, max_num_lines, suffix):
    if line_counter >= max_num_lines:
        file_counter += 1
        line_counter = 0
        output_file = create_output_file(file_counter, os.path.dirname(output_file))
    with open(output_file, 'a') as outfile:
        prompt = (
            f'--prompt "score_9, score_8_up, score_7_up, volumetric lighting, '
            f'{line.strip()}, {suffix}" '
            '--negative_prompt "score_6_up, score_5_up, score_4_up, '
            'simple background, blurry, grayscale"\n'
        )
        outfile.write(prompt)
    return line_counter + 1, file_counter, output_file

In [15]:

working_folder = r'D:\Project\CivitAI\DC\batman\draft'
out_path = r'D:\Project\CivitAI\DC\batman\sampling'
suffix  = "<lora:batman_pony_v1:0.9>"
max_num_lines = 12

os.makedirs(out_path, exist_ok=True)
process_directory(working_folder, max_num_lines, out_path = out_path, suffix=suffix)